In [2]:
import pandas as pd
import numpy as np
import nltk
import torch
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import  WordNetLemmatizer
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from transformers import pipeline
import keras
import keras.backend as K
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Dropout, Dense,GlobalMaxPooling1D,Flatten, LSTM, Bidirectional,Embedding,Layer,GlobalAveragePooling1D
from keras.layers import Input, Dense, SimpleRNN
from keras.models import Sequential
from keras import Model
import gensim
from gensim.models import *
from gensim.models.keyedvectors import KeyedVectors
from collections import Counter
from rouge_score import rouge_scorer
from transformers import pipeline
import heapq
import re
import logging
import evaluate

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
class EXSummarization():
  def __init__(self):
    pass
  def get_rouge_score(self, actual_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(actual_summary, generated_summary)
    return scores

def evaluate_abstractive(self, dataset, metric, summarizer):
    summaries =  [summarizer(text, max_length = 120, min_length = 80, do_sample = False)[0]['summary_text'] for text in dataset["dialogue"]]
    score =  metric.compute(predictions=summaries, references=dataset["highlights"])
    rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
    return rouge_dict

In [5]:
AB_summarizer = EXSummarization()

ABSTRACTIVE MODEL

In [6]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [8]:
with open('text.txt', 'r') as file:
    content = file.read()
    print(content)



**Title: The Wonders of Technology**

Technology has revolutionized the way we live, work, and communicate. From the invention of the wheel to the rise of the internet, each technological advancement has brought about significant changes in society. Today, we are witnessing rapid developments in artificial intelligence, machine learning, and robotics, which are poised to transform various industries.

One of the most exciting areas of technology is artificial intelligence (AI). AI has the potential to automate tasks, provide insights through data analysis, and even create art. Machine learning, a subset of AI, involves training algorithms on large datasets to make predictions or decisions without being explicitly programmed. This has applications in fields such as healthcare, finance, and transportation.

**Title: A Day in the Life of a Robot**

In a small town, there was a robot named Robo who was designed to help with household chores. Every morning, Robo would wake up and start it

In [9]:
summarizer(content, max_length = 120, min_length = 50, do_sample = False)

[{'summary_text': ' Technology has revolutionized the way we live, work, and communicate . Machine learning, a subset of AI, involves training algorithms on large datasets to make predictions or decisions without being explicitly programmed . This has applications in fields such as healthcare, finance, and transportation .'}]

In [10]:
from datasets import load_dataset
dataset = load_dataset("multi_news")

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [11]:
model_checkpoint = "t5-small"
max_input_length = 900
max_target_length = 120

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False, model_max_length = max_input_length)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
def preprocess_function(examples):

    model_inputs = tokenizer(
       examples["document"], max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
dataset.shuffle()

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [14]:
dataset_sample = dataset
dataset_sample['train'] = dataset['train'].shard(num_shards=50, index=0)
dataset_sample['validation'] = dataset['validation'].shard(num_shards=40, index=0)
dataset_sample['test'] = dataset['test'].shard(num_shards=40, index=0)
dataset_sample

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 141
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 141
    })
})

In [15]:
tokenized_datasets = dataset_sample.map(preprocess_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/141 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:

batch_size = 8
num_train_epochs = 5
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]
logging_steps = max(len(tokenized_datasets["train"]) // batch_size, 1)

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-Abstractive-Summarizer",
    evaluation_strategy="epoch",
    learning_rate=5.6e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
from datasets import load_metric
rouge_score = load_metric("rouge",trust_remote_code=True)

<ipython-input-18-0c6e27b98f59>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_score = load_metric("rouge",trust_remote_code=True)


In [23]:
from huggingface_hub import notebook_login
notebook_login()

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [21]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
tokenized_datasets = tokenized_datasets.remove_columns(
    dataset_sample["train"].column_names)

features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[  868, 18499,  2163,  ...,     0,     0,     0],
        [ 2922,  3441,    15,  ...,     3,  5833,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[    3,   104,    37, 17646,  1080,  6292,    12,  4848,  5406,   336,
           847,     6,    68,     8,  2717,   163,   974,   209, 13922,  2476,
             6,   116,   460, 11212,   126,  2476,   141,   118, 15439,     6,
          1315,    12,   469,    31,     7,  2476,   934,     5,   419,  4787,
            30,     8,  3556,  1887,  3559,    31,     7,  3611,  2703,   144,
          5039,    47, 18642,    10,    96,   518,    32,     9,   107,  3158,
          3862,   381,   535,    37, 17646,  1080,     6,   983,     6,    19,
           394,  1506,   117,    34,   141,   118,  1644,    12,  1520, 11207,
            44,  4848,  5170,     5,   299,     8,     3,  2965,  3358,    24,
             8, 10823,    19,  3323,   788,    12,    72,  

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)

In [25]:
import torch
torch.cuda.empty_cache()

In [26]:
from accelerate import Accelerator
accelerator = Accelerator()

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.118000,2.767724,15.134300,4.771200,11.881200,13.386000
2,2.785700,2.760872,15.764100,4.870500,12.095500,13.977900
3,2.615800,2.749421,15.151500,4.452300,11.714700,13.418100
4,2.496200,2.774270,15.344000,5.107300,12.157400,13.791700
5,2.430400,2.773668,15.703200,5.243300,12.282000,14.094600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=565, training_loss=2.6859714221110385, metrics={'train_runtime': 476.0158, 'train_samples_per_second': 9.453, 'train_steps_per_second': 1.187, 'total_flos': 1070574796800000.0, 'train_loss': 2.6859714221110385, 'epoch': 5.0})

In [28]:
trainer.evaluate()

{'eval_loss': 2.773668050765991,
 'eval_rouge1': 15.7032,
 'eval_rouge2': 5.2433,
 'eval_rougeL': 12.282,
 'eval_rougeLsum': 14.0946,
 'eval_runtime': 15.4101,
 'eval_samples_per_second': 9.15,
 'eval_steps_per_second': 1.168,
 'epoch': 5.0}

In [29]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

events.out.tfevents.1725689422.fc36e533de6d.1273.1:   0%|          | 0.00/562 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1725688930.fc36e533de6d.1273.0:   0%|          | 0.00/9.70k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MK-5/t5-small-Abstractive-Summarizer/commit/3ad340d7053130173f33f450e74e86e7d50e51b0', commit_message='Training complete', commit_description='', oid='3ad340d7053130173f33f450e74e86e7d50e51b0', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
from transformers import pipeline
hub_model_id = "MK-5/t5-small-Abstractive-Summarizer"
summarizer = pipeline("summarization", model=hub_model_id)

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [31]:
article = """SEATTLE — Seattle on Sunday set two new hot weather records as the high temperature of 95 degrees marked the sixth consecutive day in a row with high temperatures at 90 degrees or above.

The region also shattered the record of 94 degrees for this day that was set in 2015, according to the National Weather Service.

But the week-long heatwave is expected to come to an end Monday when the weather is expected to be noticeably cooler.
As of 2:30 p.m. Sunday, the high temperature at Seattle-Tacoma International Airport was 91 degrees before the mercury ultimately topped out at 95 degrees, besting by one degree the daily record set seven years ago.

"We have gotten there," she said about reaching the daily record milestone.
It was even hotter in interior parts of Washington with several areas logging triple-digit temperatures for the day.

Sunday marked the eighth time this summer that Seattle has seen a high temperature of 90 degrees or above and the fifth time since 2015 the region has logged a season with eight days or more above that threshold, according to local meteorologists.

The high temperature Saturday topped out at 94 degrees at SeaTac Airport, the fifth days of temperatures in the 90 range.

But a big cooling trend is on the way.
Stevenson said the high temperature Monday and Tuesday is expected to climb only to the low 80s with a bigger cool down expected Wednesday, which could also see drizzle.

The heat has taken a toll on the Pacific Northwest.

Authorities in the Portland announced that they would keep cooling shelters open through Sunday night.

At least seven people are suspected to have died from hyperthermia since the hot spell began a week ago in the Portland area
"""
article

'SEATTLE — Seattle on Sunday set two new hot weather records as the high temperature of 95 degrees marked the sixth consecutive day in a row with high temperatures at 90 degrees or above.\n\nThe region also shattered the record of 94 degrees for this day that was set in 2015, according to the National Weather Service.\n\nBut the week-long heatwave is expected to come to an end Monday when the weather is expected to be noticeably cooler.\nAs of 2:30 p.m. Sunday, the high temperature at Seattle-Tacoma International Airport was 91 degrees before the mercury ultimately topped out at 95 degrees, besting by one degree the daily record set seven years ago.\n\n"We have gotten there," she said about reaching the daily record milestone.\nIt was even hotter in interior parts of Washington with several areas logging triple-digit temperatures for the day.\n\nSunday marked the eighth time this summer that Seattle has seen a high temperature of 90 degrees or above and the fifth time since 2015 the re

In [32]:
articlesummary = summarizer("summarize: "+ article, max_length = 120, min_length = 80, do_sample = False)[0]["summary_text"]
articlesummary

'the high temperature at Seattle-Tacoma International Airport topped out at 95 degrees, besting by one degree the daily record set seven years ago. But a big cooling trend is on the way with a bigger cool down expected Wednesday, which could also see drizzle. At least seven people are suspected to have died from hyperthermia since the hot spell began a week ago in the Portland area. The heat has taken a toll on the Pacific Northwest.'